In [16]:
# libraries
import random
import numpy as np
import pickle
import json
from flask import Flask, render_template, request
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import random

random.seed(10)


# chat initialization
model = load_model("model_output/chatbot_model.h5")
intents = json.loads(open("data/intents.json").read())
words = pickle.load(open("model_output/words.pkl", "rb"))
classes = pickle.load(open("model_output/classes.pkl", "rb"))

app = Flask(__name__)
# run_with_ngrok(app) 

@app.route("/")
def home():
    return render_template("index_v2.html")


@app.route("/get", methods=["POST"])
def chatbot_response():
    msg = request.form["msg"].lower()
    print(msg)
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


# chat functionalities
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
   
    ERROR_THRESHOLD = 0.5

    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    #sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    print(results)
    

    confidence_probability=0.6
    return_list = []

    if len(results) == 0:
        return_list.append({"intent": 'failed', "probability": '1'})
    elif results[0][1] >=  confidence_probability:
        return_list.append({"intent": classes[results[0][0]], "probability": str(results[0][1])})
    else: 
        return_list.append({"intent": 'failed', "probability": '1'})
   
    print(return_list)
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    print(tag)
    
    list_of_intents = intents_json["intents"]

    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
        else:
            result= "Sorry, I don't understand that. Please rephrase/ask another question or send an email to gradanalytics@georgetown.edu"
        
    print("Answer: ", result)
    print("-----------------------------------------------------------------------------------------------------")
    print("\n")
    return result


if __name__ == "__main__":
    app.run(port='0000')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:55501
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2022 11:44:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 11:44:08] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Dec/2022 11:44:08] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Dec/2022 11:44:08] "GET /favicon.ico HTTP/1.1" 404 -


hi
1/1 [==============================] - 0s 57ms/step


2022-12-05 11:44:10.685262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
127.0.0.1 - - [05/Dec/2022 11:44:10] "POST /get HTTP/1.1" 200 -


[[16, 0.80256796]]
[{'intent': 'greeting', 'probability': '0.80256796'}]
greeting
Answer:  Hi there, how can I help?
-----------------------------------------------------------------------------------------------------


hi
1/1 [==============================] - 0s 15ms/step


...

[[35, 0.9058166]]
[{'intent': 'wes', 'probability': '0.9058166'}]
wes
Answer:  International applicants who attended institutions outside of the United States must submit WES. You will not need to send your WES report to our school at this time; instead, you should ask WES to send the evaluation directly to you, and then you should upload the evaluation directly to the online application. An official transcript will be required if you are offered admission to the program.
-----------------------------------------------------------------------------------------------------


i wish to speak with someone
1/1 [==============================] - 0s 16ms/step


127.0.0.1 - - [05/Dec/2022 11:44:26] "POST /get HTTP/1.1" 200 -


[[1, 0.99999917]]
[{'intent': 'Appointment', 'probability': '0.99999917'}]
Appointment
Answer:  Please send us an email at gradanalaytics@georgetown.edu
-----------------------------------------------------------------------------------------------------


is there any upcoming information session/webinar
1/1 [==============================] - 0s 15ms/step


127.0.0.1 - - [05/Dec/2022 11:44:40] "POST /get HTTP/1.1" 200 -


[[18, 0.88173425]]
[{'intent': 'information_session', 'probability': '0.88173425'}]
information_session
Answer:  The Information Session generally takes place in October- November. Applicants can register for the same. Please email gradanalytics@georgetown.edu for the exam dates and signup sheet. You can view previous information session recordings and slides here: https://analytics.georgetown.edu/admissions/oh_webinars/
-----------------------------------------------------------------------------------------------------


